<a href="https://colab.research.google.com/github/paveltyukin/gb_mysql/blob/master/les_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Практическое задание по теме “Операторы, фильтрация, сортировка и ограничение”

#### 1. Пусть в таблице users поля created_at и updated_at оказались незаполненными. Заполните их текущими датой и временем.

```
update vk.posts set created_at = now();
update vk.posts set updated_at = now();
```

#### 2. Таблица users была неудачно спроектирована. Записи created_at и updated_at были заданы типом VARCHAR и в них долгое время помещались значения в формате "20.10.2017 8:10". Необходимо преобразовать поля к типу DATETIME, сохранив введеные ранее значения.

```
alter table vk.posts add column created_ad_add varchar(255) default '20.10.2017 8:10';
select str_to_date(created_ad_add, '%d.%m.%y %h:%i') from vk.posts;
update vk.posts set created_ad_add = str_to_date(created_ad_add, '%d.%m.%y %h:%i');
alter table vk.posts modify column created_ad_add datetime
alter table vk.posts drop column created_ad_add;
```




#### 3. В таблице складских запасов storehouses_products в поле value могут встречаться самые разные цифры: 0, если товар закончился и выше нуля, если на складе имеются запасы. Необходимо отсортировать записи таким образом, чтобы они выводились в порядке увеличения значения value. Однако, нулевые запасы должны выводиться в конце, после всех записей.

![alt text](https://drive.google.com/uc?export=download&id=1CHSnbTl2rp3fdrJdJTyRklyOt4-sw45H)




```
select * from storehouses_products 
order by 
    case when value = 0 then 1 
    else 0 end, 
value asc;
```





```
SELECT 
    * 
FROM 
    storehouses_products 
ORDER BY  
    IF(value > 0,0,1),
    value;
```





```
SELECT 
    * 
FROM 
    storehouses_products 
ORDER BY  
    value = 0, value;
```



#### 4. (по желанию) Из таблицы users необходимо извлечь пользователей, родившихся в августе и мае. Месяцы заданы в виде списка английских названий ('may', 'august')





```
select 
  user_id, 
  case
    when month(birthday) = 5 then 'may'
    when month(birthday) = 8 then 'august'
  end month,
  month(birthday) 
  from vk.profiles where month(birthday) = 5 or month(birthday) = 8;
```


```
SELECT 
    name
FROM
    users
WHERE
    DATE_FORMAT(birthday_at, '%M') IN ('may', 'august');
```





#### 5. (по желанию) Из таблицы catalogs извлекаются записи при помощи запроса. SELECT * FROM catalogs WHERE id IN (5, 1, 2); Отсортируйте записи в порядке, заданном в списке IN.



```
SELECT
    *
FROM
    catalogs
WHERE
    id IN (5, 1, 2)
ORDER BY
    FIELD(id, 5, 1, 2);
```



# Практическое задание теме “Агрегация данных”

#### 1. Подсчитайте средний возраст пользователей в таблице users




```
select avg(datediff(now(), birthday))/365.25 from vk.profiles
```



```
SELECT
    AVG(TIMESTAMPDIFF(YEAR, birthday_at, NOW())) AS age
FROM
    users;
```



#### 2. Подсчитайте количество дней рождения, которые приходятся на каждый из дней недели. Следует учесть, что необходимы дни недели текущего года, а не года рождения.




```
select 
 count(*) as total, 
 week(str_to_date(concat('2020,',month(birthday),',',day(birthday)), '%Y,%m,%d')) as doy  
 from vk.profiles 
 group by doy 
 order by total desc;
```



```
SELECT
    DATE_FORMAT(DATE(CONCAT_WS('-', YEAR(NOW()), MONTH(birthday_at), DAY(birthday_at))), '%W')  AS dob, COUNT(*) AS total
FROM
    users
GROUP BY
    dob
ORDER BY
    total DSC;
```





#### 3. (по желанию) Подсчитайте произведение чисел в столбце таблицы

![alt text](https://drive.google.com/uc?export=download&id=1y15-vU7VTOFg-KwWcUX6OfO7Ps-4ejAU)






```
drop table if exists profiles_temp;

create temporary table profiles_temp
	select user_id 
    from vk.profiles 
    where user_id between 1 and 5;

select round(exp(sum(ln( user_id)))) from vk.profiles_temp;
```

